# Подготовка

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import os

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
# Эта часть нужна для удобной загрузки файлов с диска, ее можно заменить или убрать, если файлы уже скачаны
# Удобно работает в Google Colab

# Загрузка данных с SDO
!gdown https://drive.google.com/uc?id=1-iWFrKcWeLZpKtCuBJW_DlWOnMA2_8Zm
!unzip -q SDO_data.zip

# Загрузка данных GEC
!gdown https://drive.google.com/uc?id=17gAxWjAHRBNuWtG4CmgmZTjiGe_QhEEL

Downloading...
From (original): https://drive.google.com/uc?id=1-iWFrKcWeLZpKtCuBJW_DlWOnMA2_8Zm
From (redirected): https://drive.google.com/uc?id=1-iWFrKcWeLZpKtCuBJW_DlWOnMA2_8Zm&confirm=t&uuid=540937ed-b69e-4d27-a2a7-0f5665d00c5f
To: /content/SDO_data.zip
100% 821M/821M [00:11<00:00, 69.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=17gAxWjAHRBNuWtG4CmgmZTjiGe_QhEEL
To: /content/gec.csv
100% 2.70M/2.70M [00:00<00:00, 21.5MB/s]


In [ ]:
import random

# Фиксирование зерна для случайностей, чтобы работа была воспроизводимой
def set_random_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

set_random_seeds(37)

# Предобработка данных

Обработаем и приведем к нужному нейронной сети виду данные

In [ ]:
# Обработка данных GEC
df = pd.read_csv("gec.csv", sep=r'\s+')
df = df[df['Hour'] == 12]
df["timestamp"] = pd.to_datetime(df[["Year", "Month", "Day", "Hour"]], utc=True)

df = df[["timestamp", "igsg"]].rename(columns={"igsg": "gec"})
df = df[df["timestamp"].between("2014-01-01", "2024-12-31")].reset_index(drop=True)

In [ ]:
# torch.Dataset для работы с данными
# Для экономии оперативной памяти, изображения не будут загружены изначально, а при их вызове будут обрабатываться и возвращаться с диска
class GECImageDataset(Dataset):
    def __init__(self, df: pd.DataFrame, image_dir: str, wavelengths=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.wavelengths = wavelengths or ["171", "193", "211", "304", "335"]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        ts = row["timestamp"]
        gec = row["gec"]

        ts_str = pd.to_datetime(ts).strftime("%Y%m%d_%H%M%S")

        channels = []
        for wl in self.wavelengths:
            img_path = os.path.join(self.image_dir, f"{ts_str}_AIA_{wl}.jpg")
            if not os.path.exists(img_path):
                raise FileNotFoundError(f"Missing file: {img_path}")
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # (H, W)
            img = cv2.resize(img, (256, 256))
            img = img.astype(np.float32) / 255.0 # Нормализация
            channels.append(img)

        image = np.stack(channels, axis=0)  # (5, 256, 256)
        image = torch.tensor(image, dtype=torch.float32)
        label = torch.tensor([gec], dtype=torch.float32)

        return image, label


Разделение на обучающую, валидационную и тестовую выборки в пропорции 60/20/20

In [ ]:
# Предварительно перемешаем
df_shuffled = df.sample(frac=1.0, random_state=37).reset_index(drop=True)

df_train, df_temp = train_test_split(df_shuffled, test_size=0.4, random_state=37)
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=37)

print("Размер данных (наблюдений Солнца)")
print(f"Train: {len(df_train)} | Val: {len(df_val)} | Test: {len(df_test)}")

Размер данных (наблюдений Солнца)
Train: 2410 | Val: 803 | Test: 804


In [ ]:
# Датасеты и даталоадеры
train_dataset = GECImageDataset(df_train, image_dir="raw_images")
val_dataset   = GECImageDataset(df_val,   image_dir="raw_images")
test_dataset  = GECImageDataset(df_test,  image_dir="raw_images")

BATCH_SIZE = 32

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Обучение нейронной сети

Сначала создадим класс нейросети и зададим ее архитектуру

In [ ]:
class GECNet(nn.Module):
    def __init__(self):
        super(GECNet, self).__init__()

        self.conv_block = nn.Sequential(                 # Вход [5, 256, 256]
            nn.Conv2d(5, 32, kernel_size=3, padding=1),  # → [32, 256, 256]
            nn.ReLU(),
            nn.MaxPool2d(2),                             # → [32, 128, 128]

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # → [64, 128, 128]
            nn.ReLU(),
            nn.MaxPool2d(2),                             # → [64, 64, 64]

            nn.Conv2d(64, 128, kernel_size=3, padding=1),# → [128, 64, 64]
            nn.ReLU(),
            nn.MaxPool2d(2),                             # → [128, 32, 32]
        )

        self.fc = nn.Sequential(
            nn.Flatten(),                                # → [128 * 32 * 32]
            nn.Linear(128 * 32 * 32, 256),
            nn.ReLU(),
            nn.Dropout(0.3),                             # Для предотвращения переобучения
            nn.Linear(256, 1)                            # → [1] — регрессия GEC
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc(x)
        return x

In [ ]:
#@title Функции для обучения, валидации и инференса модели

import copy

# Функция для одного шага обучения
def train_step(batch, model, loss, optimizer, device, scheduler=None):

    X, y = batch
    X = X.to(device)
    y = y.to(device)

    preds = model(X)
    l = loss(preds, y)

    l.backward()
    optimizer.step()
    if scheduler != None:
      scheduler.step()

    optimizer.zero_grad()

    return l.item()

# Функция для обучения на эпохе.
def train(model, loss, optimizer, device, train_dataloader, scheduler=None, verbose=1):
    model.train()
    train_loss = 0

    for batch in (tqdm(train_dataloader, desc="Обучение") if verbose <= 1 else train_dataloader):
      loss_step = train_step(batch, model, loss, optimizer, device, scheduler)
      train_loss += loss_step / len(train_dataloader)

    return train_loss

# Функция для одного шага валидации
def valid_step(batch, model, loss, device):

      X, y = batch
      X = X.to(device)
      y = y.to(device)

      with torch.no_grad():
        preds = model(X)
        l = loss(preds, y)

      return preds.detach().cpu().numpy(), l.item()

# Функция для всей валидации на эпохе
def validate(model, loss, device, val_dataloader, verbose=1):
  model.eval()
  val_loss = 0
  preds = []
  for batch in (tqdm(val_dataloader, desc="Валидация") if verbose <= 1 else val_dataloader):
    preds_step, loss_step = valid_step(batch, model, loss, device)

    val_loss += loss_step / len(val_dataloader)
    preds.append(preds_step)

  preds = np.concatenate(preds)

  return preds, val_loss

# Объединяем обучение и валидацию. Модификация функции под свои нужды
def train_and_validate(epochs, model, loss, optimizer, device, train_dataloader, val_dataloader, patience=20, verbose=5, scheduler=None):
    model.to(device)
    best_loss = float('inf')
    best_model = None
    counter = 0
    train_losses = []
    val_losses = []

    for e in (range(epochs) if verbose <= 1 else tqdm(range(epochs), desc="Цикл обучения")):
        train_loss = train(model, loss, optimizer, device, train_dataloader, scheduler, verbose)
        val_preds, val_loss = validate(model, loss, device, val_dataloader, verbose)
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        if e % verbose == 0:
            print(f'Эпоха: {e} | Train Loss {train_loss} | Val Loss {val_loss}')

        if val_loss < best_loss:
            best_loss = val_loss
            counter = 0
            best_model = copy.deepcopy(model.state_dict())
        else:
            counter += 1
        if counter >= patience:
            print(f"\nРанняя остановка на эпохе {e}")
            break

    model.load_state_dict(best_model)
    return model, train_losses, val_losses

# Один шаг инференса
def inference_step(batch, model, device):
    X, _ = batch
    X = X.to(device)
    with torch.no_grad():
        preds = model(X)
    return preds.detach().cpu().numpy()

# Полный инференс
def inference(model, device, dataloader, verbose=1):
    model.eval()
    preds = []
    for batch in (tqdm(dataloader, desc="Инференс") if verbose <= 1 else dataloader):
        preds_step = inference_step(batch, model, device)
        preds.append(preds_step)

    preds = np.concatenate(preds)
    return preds


Далее создаем и обучаем модель

In [ ]:
# Параметры модели и обучения

epochs = 50
# Количество эпох с запасом для механизма ранней остановки (откат к лучшей эпохе, если ошибка не уменьшается)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GECNet()

loss = nn.L1Loss()
# MAE

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# Классический оптимизатор Adam

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=1e-3,
    steps_per_epoch=len(train_loader),
    epochs=epochs,
    anneal_strategy='cos'
)
# Шедулер для оптимизации скорости обучения (lr)

In [ ]:
# Обучение
model, train_losses, val_losses = train_and_validate(epochs=epochs,
                               model=model,
                               loss=loss,
                               optimizer=optimizer,
                               scheduler=scheduler,
                               device=device,
                               train_dataloader=train_loader,
                               val_dataloader=val_loader,
                               patience=15,
                               verbose=1)

Валидация: 100%|██████████| 26/26 [00:08<00:00,  3.15it/s]


Эпоха: 0 | Train Loss 0.30227345610527623 | Val Loss 0.14638105178108582


Валидация: 100%|██████████| 26/26 [00:07<00:00,  3.69it/s]


Эпоха: 1 | Train Loss 0.16530426875933218 | Val Loss 0.12383755181844416


Валидация: 100%|██████████| 26/26 [00:07<00:00,  3.66it/s]


...

Эпоха: 47 | Train Loss 0.053840400896182194 | Val Loss 0.05247104769715896


Валидация: 100%|██████████| 26/26 [00:05<00:00,  4.37it/s]


Эпоха: 48 | Train Loss 0.05544450207564392 | Val Loss 0.05234059906349732


Валидация: 100%|██████████| 26/26 [00:05<00:00,  4.49it/s]

Эпоха: 49 | Train Loss 0.05481196121361696 | Val Loss 0.052193437416393026


Сохраняем лучшие веса модели

In [ ]:
torch.save(model.state_dict(), "gec_model_weights.pth")

# Тестирование и визуализация

In [ ]:
# Тестирование модели

test_preds = inference(model, device, test_loader)
y_test_true = np.concatenate([y.numpy() for _, y in test_dataset])
test_mae = mean_absolute_error(y_test_true, test_preds)
print(f"MAE на тестовых данных: {test_mae}")

Инференс: 100%|██████████| 26/26 [00:06<00:00,  4.03it/s]


MAE на тестовых данных: 0.053017109632492065


In [ ]:
# Предсказание на всех данных модели

all_dataset = GECImageDataset(df, image_dir="raw_images")
all_loader = DataLoader(all_dataset, batch_size=BATCH_SIZE, shuffle=False,  num_workers=2)

all_preds = inference(model, device, all_loader)
y_all_true = np.concatenate([y.numpy() for _, y in all_loader])
all_mae = mean_absolute_error(y_all_true, all_preds)
print(f"MAE на всех данных: {all_mae}")

Инференс: 100%|██████████| 126/126 [00:32<00:00,  3.86it/s]


MAE на всех данных: 0.035563379526138306


Визуализации: отклонение предсказаний от истинных значений, график качества обучения, GEC в зависимости от даты

In [ ]:
#@title Отклонение предсказаний

data = pd.DataFrame({
    "True": y_test_true,
    "Predicted": test_preds.squeeze()
})

fig = px.scatter(data, x="True", y="Predicted", opacity=0.8,
                 labels={
                     "True": "Истинное значение GEC",
                     "Predicted": "Предсказанное значение GEC"
                 },
                 title="Отклонение предсказаний от истинных значений GEC на тестовых данных",
                 width=800, height=800)
fig.add_shape(
    type="line",
    x0=data["True"].min(), y0=data["True"].min(),
    x1=data["True"].max(), y1=data["True"].max(),
    line=dict(color="red", dash="dash")
)
fig.show()

In [ ]:
#@title Качество обучения

data = pd.DataFrame({
    "Epoch": range(len(train_losses)),
    "Train": train_losses,
    "Validation": val_losses
})

fig = px.line(data, x="Epoch", y=["Train", "Validation"],
              title="Средняя абсолютная ошибка обучения по эпохам",
              labels={
                  "Value": "Значение MAE",
                  "Epoch": "Эпоха обучения"
              },
              markers=True)
fig.show()


In [ ]:
#@title График GEC

df_plot = df.copy()
df_plot["Predicted GEC"] = all_preds
df_plot = df_plot.rename(columns={"timestamp": "Date", "gec": "True GEC"})

fig = px.line(df_plot, x="Date", y=["True GEC", "Predicted GEC"],
              title="Истинный и предсказанный GEC по времени",
              labels={"value": "GEC", "Date": "Дата"})

fig.show()

# Анализ важности каналов

Функция, которая поочередно заглушает каждый канал (заменяет все значения на 0) и фиксирует прирост MAE

In [ ]:
def evaluate_channel_importance(model, device, dataset, original_mae):
    model.eval()
    y_true = np.concatenate([y.numpy() for _, y in dataset]).reshape(-1)
    results = []

    for ch in range(5):
        print(f"Заглушаем канал {ch}...")

        class MaskedDataset(Dataset):
            def __init__(self, base_dataset, channel_to_mask):
                self.base = base_dataset
                self.ch = channel_to_mask

            def __len__(self): return len(self.base)

            def __getitem__(self, idx):
                x, y = self.base[idx]
                x = x.clone()
                x[self.ch, :, :] = 0.0
                return x, y

        masked_dataset = MaskedDataset(dataset, ch)
        loader = DataLoader(masked_dataset, batch_size=32)
        y_pred_masked = inference(model, device, loader).reshape(-1)

        mae_masked = mean_absolute_error(y_true, y_pred_masked)
        delta = mae_masked - original_mae

        results.append({
            "channel": ch,
            "MAE": mae_masked,
            "Delta_MAE": delta
        })

    return results

In [ ]:
# Запускаем
importance_results = evaluate_channel_importance(model, device, all_dataset, all_mae)

# Преобразуем в DataFrame
df_imp = pd.DataFrame(importance_results)
df_imp["wavelength"] = ["0171", "0193", "0211", "0304", "0335"]

# Столбчатая диаграмма ухудшения
fig = px.bar(df_imp, x="wavelength", y="Delta_MAE", title="Вклад длин волн (Ablation Study)",
             labels={"Delta_MAE": "Рост MAE при заглушке", "wavelength": "Длина волны (Å)"})
fig.show()

Заглушаем канал 0...


Инференс: 100%|██████████| 126/126 [00:41<00:00,  3.06it/s]


Заглушаем канал 1...


Инференс: 100%|██████████| 126/126 [00:39<00:00,  3.15it/s]


Заглушаем канал 2...


Инференс: 100%|██████████| 126/126 [00:41<00:00,  3.03it/s]


Заглушаем канал 3...


Инференс: 100%|██████████| 126/126 [00:39<00:00,  3.16it/s]


Заглушаем канал 4...


Инференс: 100%|██████████| 126/126 [00:39<00:00,  3.15it/s]


# Прогнозирование

In [ ]:
def predict_gec(model, image_dir, timestamp, device, wavelengths=["171", "193", "211", "304", "335"]):
    model.eval()
    ts_str = pd.to_datetime(timestamp).strftime("%Y%m%d_%H%M%S")

    channels = []
    for wl in wavelengths:
        img_path = os.path.join(image_dir, f"{ts_str}_AIA_{wl}.jpg")
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Missing file: {img_path}")
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (256, 256))
        img = img.astype(np.float32) / 255.0  # нормализация 0–1
        channels.append(img)

    x = np.stack(channels, axis=0)                # (5, 256, 256)
    x = torch.tensor(x, dtype=torch.float32).unsqueeze(0).to(device)  # → (1, 5, 256, 256)

    with torch.no_grad():
        pred = model(x).cpu().item()

    return pred


Чтобы получить предсказание, необходимо привести изображения к виду: YYmmdd_HHMMSS_AIA_000.jpg, где 000 это код канала.

При желании можно загрузить веса модели и пропустить обучение. Для этого в директории со скриптом должен быть файл gec_model_weights.pth



In [ ]:
timestamp = "2023-06-15 12:00:00" # Пример
image_dir = "raw_images" # Пример
load_weights = True # Загрузить или не загружать веса

if load_weights:
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = GECNet()
  model.load_state_dict(torch.load("gec_model_weights.pth", map_location=device))
  model.to(device)
  model.eval()


gec_pred = predict_gec(model, image_dir, timestamp, device)
print(f"Предсказанный GEC: {gec_pred:.4f}")


Предсказанный GEC: 1.2444
